Тестовое задание
================

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import pandas as pd
pd.set_option('display.max_rows', None)

## Pandas

Файл `task_history.csv` содержит исторические данные системы сборки (e.g. *jenkins*, *teamcity*, etc.)

Каждая запись в истории описывает отдельный запуск задачи `build_name` на рабочей машине `worker`.<br/>
Для любого запуска известны автор `author` и проект `project`, в рамках которого был запущен расчет данной задачи.

In [ ]:
df = pd.read_csv('task_history.csv', parse_dates=['start_time', 'finish_time'])

df['date'] = df['start_time'].dt.date
df['duration'] = (df.finish_time - df.start_time).dt.total_seconds() / 3600
df['week'] = df['start_time'].dt.isocalendar().week
df['week'] -= df['week'].min() - 1
df['weekday'] = df['start_time'].dt.dayofweek
df['day'] = df['start_time'].dt.strftime('%m-%d')  

### Задача 1

Для каждой недели распечатайте топ-3 пользователей, потребивших наибольшее количество машиночасов.

Выведите для данных пользователей процент потребленных ресурсов от общего объема расчетов за эту неделю.

In [ ]:
total_time = df.groupby('week')['duration'].sum()
week_consumption = (df.groupby(['week', 'author'])['duration'].sum() / total_time * 100).round(2)
result = week_consumption.groupby('week').nlargest(3, keep='all').reset_index(level=0, drop=True)
result.rename('DurationInPercent', inplace=True)
result = result.astype(str) + '%'
print(result)

### Задача 2

Отобразите график с ежедневным потреблением машинного времени каждым проектом.

In [ ]:
grouped = (df.groupby(['day', 'project'])['duration'].sum()).round(2)
ax = grouped.unstack(level=1).plot(kind='line', ylabel='total hours', xlabel='date', rot=0, title='total hours per project')
ax.xaxis.set_major_locator(dates.DayLocator(interval=1))
    
plt.show()

### Задача 3

Рассмотрим *еженедельное* общее потребление ресурсов. На второй неделе можно заметить всплеск нагрузки с ее последующим планомерным снижением.

Проанализируйте предоставленные данные и найдите, с чем связано подобное поведение.

In [ ]:
# Выводы по заданию здесь, код ниже:
# Нагрузка имеет сезонность, падает к концу недели, чаще субботе, после чего рост.
# 
# Еженедельное потребление
df.groupby('week')['duration'].sum().plot()

## Python

## Задача 4
Слияние логов. <br>
Имеется два файла с логами в формате JSONL, пример лога:
```
{"timestamp": "2021-02-26 08:59:20", "log_level": "INFO", "message": "Hello"}
{"timestamp": "2021-02-26 09:01:14", "log_level": "INFO", "message": "Crazy"}
{"timestamp": "2021-02-26 09:03:36", "log_level": "INFO", "message": "World!"}
```
Сообщения в заданных файлах упорядочены по полю timestamp в порядке возрастания.<br>
Требуется написать функцию, которая объединит эти два файла в один новый файл.<br>
При этом сообщения в получившемся файле тоже должны быть упорядочены в порядке возрастания по полю `timestamp`.
К заданию прилагается вспомогательный скрипт на `python3`, который создает два файла "log_a.jsonl" и "log_b.jsonl".<br>
Командлайн для запуска:
```
python log_generator.py <path/to/dir>
```

In [ ]:
import json


class Merger:
    def __init__(self, file1, file2, output):
        self.file1_handler = open(file1, 'r')
        self.file2_handler = open(file2, 'r')
        self.output_handler = open(output, 'w')
        self.file1_line = self.file1_handler.readline()
        self.file2_line = self.file2_handler.readline()
    

    def merge_two(self):
        while True:
            if json.loads(self.file1_line)['timestamp'] < json.loads(self.file2_line)['timestamp']:
                self.output_handler.write(self.file1_line)
                self.file1_line = self.file1_handler.readline()

                if not self.file1_line:
                    first_handler = False
                    break
            else:
                self.output_handler.write(self.file2_line)
                self.file2_line = self.file2_handler.readline()

                if not self.file2_line:
                    first_handler = True
                    break

        self.merge_one(first_handler=first_handler)
        

    def merge_one(self, first_handler):
        if first_handler:
            while self.file1_line:
                self.output_handler.write(self.file1_line)
                self.file1_line = self.file1_handler.readline()
        else:
            while self.file2_line:
                self.output_handler.write(self.file2_line)
                self.file2_line = self.file2_handler.readline()

        self.close()
        
    def close(self):
        self.file1_handler.close()
        self.file2_handler.close()
        self.output_handler.close()

        print("Merging of JSONL files is completed.")
        return


file1_path = "./logs/log_a.jsonl"
file2_path = "./logs/log_b.jsonl" 
output_path = "./logs/merged_file.jsonl"


obj = Merger(file1_path, file2_path, output_path)
obj.merge_two()